In [ ]:
import random
import pandas as pd
import numpy as np

from datetime import datetime
from datetime import timedelta

from IPython.display import clear_output

from timeit import default_timer as timer



def simulador(dataframe: pd.DataFrame, linea: dict, meses, capacidad_tren: int, frecuencia: int,
              sesgo_distancia: float, nivel_sesgo_distancia: int, nivel_sesgo_siguiente: int,
              porcentaje_sin_pagar: float, probabilidad_no_subir: float,
              incremento_probabilidad_no_subir: list, penalizacion_llenado_imperfecto: float,
              dia_semana: str):
    class Linea:
        def __init__(self, nombre: str, estaciones: list, recorrido_A: dict, recorrido_B: dict, 
        horario_apertura_cabecera_A: str, 
        horario_cierre_cabecera_A: str, horario_apertura_cabecera_B: str, 
        horario_cierre_cabecera_B: str, combinaciones: dict, capacidad_tren: int,
        frecuencia: int, sesgo_distancia: float, nivel_sesgo_distancia: int, 
        nivel_sesgo_siguiente: int, porcentaje_sin_pagar: float, probabilidad_no_subir: float,
        incremento_probabilidad_no_subir: list, penalizacion_llenado_imperfecto: float):
            self.nombre = nombre
            self.estaciones = estaciones
            self.recorrido_A = recorrido_A
            self.recorrido_B = recorrido_B
            self.horario_apertura_cabecera_A = horario_apertura_cabecera_A
            self.horario_cierre_cabecera_A = horario_cierre_cabecera_A
            self.horario_apertura_cabecera_B = horario_apertura_cabecera_B
            self.horario_cierre_cabecera_B = horario_cierre_cabecera_B
            self.combinaciones = combinaciones
            self.capacidad_tren = capacidad_tren
            self.frecuencia = frecuencia
            self.sesgo_distancia = sesgo_distancia
            self.nivel_sesgo_distancia = nivel_sesgo_distancia
            self.nivel_sesgo_siguiente = nivel_sesgo_siguiente
            self.porcentaje_sin_pagar = porcentaje_sin_pagar
            self.probabilidad_no_subir = probabilidad_no_subir
            self.incremento_probabilidad_no_subir = incremento_probabilidad_no_subir
            self.penalizacion_llenado_imperfecto = penalizacion_llenado_imperfecto

    class Estacion:
        def __init__(self, nombre: str, linea: str):
            self.nombre = nombre
            self.linea = linea
            self.anden_A = []
            self.anden_B = []

    class Tren:
        def __init__(self, id: int, linea: str, capacidad_tren: int, recorrido: list):
            self.id = id
            self.linea = linea
            self.recorrido = recorrido
            self.anden = self.recorrido[0]
            self.estacion_actual = np.nan
            self.carga = []
            self.capacidad_tren = capacidad_tren
            
        def mover(self):
            self.anden = self.recorrido[self.recorrido.index(self.anden)+1]
            
    class Pasajero:
        def __init__(self, destino: str):
            self.id = random.randint(1000000000000,9999999999999)
            self.destino = destino
        


    for i in range(0, len(linea)):
        diccionario_A = {"linea": [], "mes": [], "horario": [], "tren": [], 
                            "estacion": [], "ingresos": [], "egresos": [], "carga": [],
                            "capacidad": []}
        diccionario_B = {"linea": [], "mes": [], "horario": [], "tren": [], 
                            "estacion": [], "ingresos": [], "egresos": [], "carga": [],
                            "capacidad": []}

        for mes in (dataframe["mes"].unique() if meses=="todos" else ([meses] if isinstance(meses, int) else meses)):
            linea_mes = Linea(list(linea.keys())[i], list(dataframe[dataframe["mes"]==mes]["estacion"].unique()),
                                linea[list(linea.keys())[i]]["recorrido_A"], 
                                linea[list(linea.keys())[i]]["recorrido_B"],
                                linea[list(linea.keys())[i]]["horario_apertura_cabecera_A"],
                                linea[list(linea.keys())[i]]["horario_cierre_cabecera_A"],
                                linea[list(linea.keys())[i]]["horario_apertura_cabecera_B"],
                                linea[list(linea.keys())[i]]["horario_cierre_cabecera_B"],
                                linea[list(linea.keys())[i]]["combinaciones"],
                                capacidad_tren, frecuencia, sesgo_distancia, nivel_sesgo_distancia,
                                nivel_sesgo_siguiente, porcentaje_sin_pagar,
                                probabilidad_no_subir, incremento_probabilidad_no_subir,
                                penalizacion_llenado_imperfecto)
            
            estaciones_objeto = []
            for estacion in linea_mes.estaciones:
                estaciones_objeto.append(Estacion(estacion, linea_mes.nombre))
                
            # sentido A
            inicio_A = timedelta(hours=datetime.strptime(linea_mes.horario_apertura_cabecera_A, 
                                                            "%H:%M:%S").hour,
                                minutes=datetime.strptime(linea_mes.horario_apertura_cabecera_A, 
                                                            "%H:%M:%S").minute,
                                seconds=datetime.strptime(linea_mes.horario_apertura_cabecera_A, 
                                                            "%H:%M:%S").second)
            final_A = timedelta(hours=datetime.strptime(linea_mes.horario_cierre_cabecera_A, 
                                                        "%H:%M:%S").hour,
                                minutes=datetime.strptime(linea_mes.horario_cierre_cabecera_A, 
                                                        "%H:%M:%S").minute,
                                seconds=datetime.strptime(linea_mes.horario_cierre_cabecera_A, 
                                                        "%H:%M:%S").second)
            tiempo_A = (final_A - inicio_A).seconds//60
            
            horarios_A = []
            
            # clock
            for minuto in range(0, tiempo_A+len(linea_mes.recorrido_A)+1):
                horarios_A.append(datetime.strftime((datetime.strptime(linea_mes.horario_apertura_cabecera_A, 
                                                                        "%H:%M:%S")
                                    + timedelta(minutes=minuto)), "%H:%M:%S"))
            
            trenes_A = []
            
            contador_frecuencia = linea_mes.frecuencia
            
            
            for hora in horarios_A:
                if contador_frecuencia == linea_mes.frecuencia:
                    if datetime.strptime(hora, "%H:%M:%S") <= datetime.strptime(
                                                        linea_mes.horario_cierre_cabecera_A, "%H:%M:%S"):
                        trenes_A.append(Tren(random.randint(1000000000000,9999999999999), 
                                                linea_mes.nombre, linea_mes.capacidad_tren, 
                                                linea_mes.recorrido_A))
                        contador_frecuencia = 0
                    
                for tren in trenes_A.copy():                    
                    if tren.anden in linea_mes.estaciones: 
                        #estaciones deberia ser un dict indicando si tiene trasbordo o no
                        subida = dataframe[dataframe["mes"]==int(mes)]
                        subida = subida[subida["estacion"]==tren.anden]
                        total_pasajeros_subida = 0
                        for minuto in range(0, linea_mes.frecuencia):
                            hora_aux = datetime.strptime(hora, "%H:%M:%S") - timedelta(minutes=minuto)
                            hora_aux = datetime.strftime(hora_aux, "%H:%M:%S")
                            try:
                                pasajeros_minuto = float(subida[subida["horario"]==hora_aux][
                                                                                dataframe.columns.values[4]])
                            except:
                                pasajeros_minuto = 0
                            total_pasajeros_subida += pasajeros_minuto
                        
                        if tren.anden in linea_mes.combinaciones.keys():
                            total_pasajeros_subida *= linea_mes.combinaciones[
                                                        tren.anden]["ingreso_a_b"][0]
                        
                        total_pasajeros_subida += total_pasajeros_subida*linea_mes.porcentaje_sin_pagar
                        
                        estaciones_restantes = linea_mes.recorrido_A.copy()
                        estaciones_restantes.remove("final")
                        
                        for l in ["-0", "-1", "-2", "-3",
                                "-4","-5","-6","-7","-8",
                                "-9","-10","-11","-12","-13",
                                "-14","-15","-16","-17","-18"]:
                            try:
                                estaciones_restantes.remove(l)
                            except ValueError:
                                pass
                        
                        estaciones_restantes = estaciones_restantes[
                                                estaciones_restantes.index(tren.anden)+1:]
                        estaciones_restantes_aux = estaciones_restantes.copy()
                        for estacion in estaciones_restantes_aux:
                            if estacion in linea_mes.estaciones:
                                if estacion in linea_mes.combinaciones.keys():
                                    for _ in range(1, linea_mes.combinaciones[estacion]["egreso_a_b"][0]):
                                        estaciones_restantes.append(estacion)
                            else:
                                estaciones_restantes.remove(estacion)
                                
                        for estacion in estaciones_restantes_aux:
                            if estacion in linea_mes.estaciones:
                                if len(estaciones_restantes_aux) > 1:
                                    if estacion in estaciones_restantes_aux[int(len(estaciones_restantes_aux)*linea_mes.sesgo_distancia+0.5):]:
                                        for _ in range(0, linea_mes.nivel_sesgo_distancia):
                                            estaciones_restantes.append(estacion)

                        for estacion in estaciones_restantes.copy():
                            if estacion != estaciones_restantes_aux[0]:
                                for _ in range(0, linea_mes.nivel_sesgo_siguiente):
                                    estaciones_restantes.append(estacion)
                        
                        # bajada
                        h = 0
                        for pasajero in tren.carga.copy():
                            if pasajero.destino == tren.anden:
                                tren.carga.remove(pasajero)
                                h += 1
                        
                        # subida
                        if len(estaciones_restantes) != 0:
                            # pasajeros que esperan en el anden
                            for estacion_objeto in estaciones_objeto:
                                if estacion_objeto.nombre == tren.anden:
                                    pasajeros_espera = len(estacion_objeto.anden_A.copy())
                                    for pasajero in estacion_objeto.anden_A.copy():
                                        tren.carga.append(pasajero)
                                        estacion_objeto.anden_A.remove(pasajero)
                                    break
                            pasajeros_espera_nuevos = 0
                            for _ in range(0, int(total_pasajeros_subida+0.5)):
                                if len(tren.carga) >= (tren.capacidad_tren-tren.capacidad_tren*penalizacion_llenado_imperfecto):
                                    probabilidad_aux = linea_mes.probabilidad_no_subir + linea_mes.incremento_probabilidad_no_subir[0]*(int(len(tren.carga)-(tren.capacidad_tren-tren.capacidad_tren*penalizacion_llenado_imperfecto))*(1/linea_mes.incremento_probabilidad_no_subir[1]))
                                    if random.randint(0, 1) <= probabilidad_aux:
                                        for estacion_objeto in estaciones_objeto:
                                            if estacion_objeto.nombre == tren.anden:
                                                eleccion = random.choice(estaciones_restantes)
                                                estacion_objeto.anden_A.append(Pasajero(eleccion))
                                                pasajeros_espera_nuevos += 1
                                                break
                                        continue
                                eleccion = random.choice(estaciones_restantes)
                                tren.carga.append(Pasajero(eleccion))
                    
                        diccionario_A["linea"].append(linea_mes.nombre)
                        diccionario_A["mes"].append(mes)
                        diccionario_A["horario"].append(hora)
                        diccionario_A["tren"].append(tren.id)
                        diccionario_A["estacion"].append(tren.anden)
                        diccionario_A["ingresos"].append(int(total_pasajeros_subida+0.5)-pasajeros_espera_nuevos+pasajeros_espera)
                        diccionario_A["egresos"].append(h)
                        diccionario_A["carga"].append(len(tren.carga))
                        diccionario_A["capacidad"].append(tren.capacidad_tren)
                    tren.mover()
                    if tren.anden == "final":
                        trenes_A.remove(tren)
                    
                contador_frecuencia += 1
                
            # sentido B
            inicio_B = timedelta(hours=datetime.strptime(linea_mes.horario_apertura_cabecera_B, 
                                                            "%H:%M:%S").hour,
                                minutes=datetime.strptime(linea_mes.horario_apertura_cabecera_B, 
                                                            "%H:%M:%S").minute,
                                seconds=datetime.strptime(linea_mes.horario_apertura_cabecera_B, 
                                                            "%H:%M:%S").second)
            final_B = timedelta(hours=datetime.strptime(linea_mes.horario_cierre_cabecera_B, 
                                                        "%H:%M:%S").hour,
                                minutes=datetime.strptime(linea_mes.horario_cierre_cabecera_B, 
                                                        "%H:%M:%S").minute,
                                seconds=datetime.strptime(linea_mes.horario_cierre_cabecera_B, 
                                                        "%H:%M:%S").second)
            tiempo_B = (final_B - inicio_B).seconds//60
            
            horarios_B = []
            
            # clock
            for minuto in range(0, tiempo_B+len(linea_mes.recorrido_B)+1):
                horarios_B.append(datetime.strftime((datetime.strptime(linea_mes.horario_apertura_cabecera_B, 
                                                                        "%H:%M:%S")
                                    + timedelta(minutes=minuto)), "%H:%M:%S"))
            
            trenes_B = []
            
            contador_frecuencia = linea_mes.frecuencia
            
            for hora in horarios_B:
                if contador_frecuencia == linea_mes.frecuencia:
                    if datetime.strptime(hora, "%H:%M:%S") <= datetime.strptime(
                                        linea_mes.horario_cierre_cabecera_B, "%H:%M:%S"):
                        trenes_B.append(Tren(random.randint(1000000000000,9999999999999), 
                                                linea_mes.nombre, linea_mes.capacidad_tren, 
                                                linea_mes.recorrido_B))
                        contador_frecuencia = 0
                for tren in trenes_B.copy():
                    if tren.anden in linea_mes.estaciones: 
                        #estaciones deberia ser un dict indicando si tiene trasbordo o no
                        subida = dataframe[dataframe["mes"]==int(mes)]
                        subida = subida[subida["estacion"]==tren.anden]
                        total_pasajeros_subida = 0
                        for minuto in range(0, linea_mes.frecuencia):
                            hora_aux = datetime.strptime(hora, "%H:%M:%S") - timedelta(minutes=minuto)
                            hora_aux = datetime.strftime(hora_aux, "%H:%M:%S")
                            try:
                                pasajeros_minuto = float(subida[subida["horario"]==hora_aux][
                                                                                dataframe.columns.values[6]])
                            except:
                                pasajeros_minuto = 0
                            total_pasajeros_subida += pasajeros_minuto
                        
                        if tren.anden in linea_mes.combinaciones.keys():
                            total_pasajeros_subida *= linea_mes.combinaciones[
                                                        tren.anden]["ingreso_a_b"][1]
                        
                        total_pasajeros_subida += total_pasajeros_subida*linea_mes.porcentaje_sin_pagar
                        
                        estaciones_restantes = linea_mes.recorrido_B.copy()
                        estaciones_restantes.remove("final")
                        
                        for l in ["-0", "-1", "-2", "-3",
                                "-4","-5","-6","-7","-8",
                                "-9","-10","-11","-12","-13",
                                "-14","-15","-16","-17","-18"]:
                            try:
                                estaciones_restantes.remove(l)
                            except ValueError:
                                pass
                        
                        estaciones_restantes = estaciones_restantes[
                                                estaciones_restantes.index(tren.anden)+1:]
                        estaciones_restantes_aux = estaciones_restantes.copy()
                        for estacion in estaciones_restantes_aux:
                            if estacion in linea_mes.estaciones:
                                if estacion in linea_mes.combinaciones.keys():
                                    for _ in range(1, linea_mes.combinaciones[estacion]["egreso_a_b"][0]):
                                        estaciones_restantes.append(estacion)
                            else:
                                estaciones_restantes.remove(estacion)
                                
                        for estacion in estaciones_restantes_aux:
                            if estacion in linea_mes.estaciones:
                                if len(estaciones_restantes_aux) > 1:
                                    if estacion in estaciones_restantes_aux[int(len(estaciones_restantes_aux)*linea_mes.sesgo_distancia+0.5):]:
                                        for _ in range(0, linea_mes.nivel_sesgo_distancia):
                                            estaciones_restantes.append(estacion)
                                
                        for estacion in estaciones_restantes.copy():
                            if estacion != estaciones_restantes_aux[0]:
                                for _ in range(0, linea_mes.nivel_sesgo_siguiente):
                                    estaciones_restantes.append(estacion)

                        # bajada
                        h = 0
                        for pasajero in tren.carga.copy():
                            if pasajero.destino == tren.anden:
                                tren.carga.remove(pasajero)
                                h += 1


                
                        # subida
                        if len(estaciones_restantes) != 0:
                            # pasajeros que esperan en el anden
                            for estacion_objeto in estaciones_objeto:
                                if estacion_objeto.nombre == tren.anden:
                                    pasajeros_espera = len(estacion_objeto.anden_B.copy())
                                    for pasajero in estacion_objeto.anden_B.copy():
                                        tren.carga.append(pasajero)
                                        estacion_objeto.anden_B.remove(pasajero)
                                    break
                            pasajeros_espera_nuevos = 0
                            for _ in range(0, int(total_pasajeros_subida+0.5)):
                                if len(tren.carga) >= (tren.capacidad_tren-tren.capacidad_tren*penalizacion_llenado_imperfecto):
                                    probabilidad_aux = linea_mes.probabilidad_no_subir + linea_mes.incremento_probabilidad_no_subir[0]*(int(len(tren.carga)-(tren.capacidad_tren-tren.capacidad_tren*penalizacion_llenado_imperfecto))*(1/linea_mes.incremento_probabilidad_no_subir[1]))
                                    if random.randint(0, 1) <= probabilidad_aux:
                                        for estacion_objeto in estaciones_objeto:
                                            if estacion_objeto.nombre == tren.anden:
                                                eleccion = random.choice(estaciones_restantes)
                                                estacion_objeto.anden_B.append(Pasajero(eleccion))
                                                pasajeros_espera_nuevos += 1
                                                break
                                        continue
                                eleccion = random.choice(estaciones_restantes)
                                tren.carga.append(Pasajero(eleccion))

                                
                    
                        diccionario_B["linea"].append(linea_mes.nombre)
                        diccionario_B["mes"].append(mes)
                        diccionario_B["horario"].append(hora)
                        diccionario_B["tren"].append(tren.id)
                        diccionario_B["estacion"].append(tren.anden)
                        diccionario_B["ingresos"].append(int(total_pasajeros_subida+0.5)-pasajeros_espera_nuevos+pasajeros_espera)
                        diccionario_B["egresos"].append(h)
                        diccionario_B["carga"].append(len(tren.carga))
                        diccionario_B["capacidad"].append(tren.capacidad_tren)
                    tren.mover()
                    if tren.anden == "final":
                        trenes_B.remove(tren)
                 
                contador_frecuencia += 1
                
      
        df_trenes_A = pd.DataFrame(data=diccionario_A)
        df_trenes_A.to_csv(f"datos_finales_{list(linea.keys())[i]}_A_{dia_semana}.csv", index=False)
                
                
        df_trenes_B = pd.DataFrame(data=diccionario_B)
        df_trenes_B.to_csv(f"datos_finales_{list(linea.keys())[i]}_B_{dia_semana}.csv", index=False)
            

if __name__ == "__main__":
    def programa(dia_semana):
        
        if dia_semana in ("lunes", "martes", "miercoles", "jueves", "viernes"):
            linea = {"A": {
                "recorrido_A": ['San Pedrito', 'Flores', "-1", 'Carabobo', "-2", 
                                'Puan', "-3", 'Primera Junta', "-4",
                                'Acoyte', 'Rio de Janeiro', "-5", 'Castro Barros', 
                                "-6", 'Loria', "-7", 'Plaza Miserere', 'Pasco', 
                                "-8", 'Congreso', 'Saenz Peña', 'Lima', 'Piedras', 
                                "-9", 'Peru', 'Plaza de Mayo', "final"],
                "recorrido_B": ['Plaza de Mayo', 'Peru', 'Piedras', 'Lima', 
                                'Saenz Peña', "-1", 'Congreso', 'Alberti', "-2", 
                                'Plaza Miserere', "-3", 'Loria', 'Castro Barros', 
                                "-4", 'Rio de Janeiro', 'Acoyte', "-5", 
                                'Primera Junta', "-6", 'Puan', "-7", 'Carabobo', 
                                'Flores', 'San Pedrito', "final"],
                "horario_apertura_cabecera_A": "05:30:00", 
                "horario_cierre_cabecera_A": "23:00:00",
                "horario_apertura_cabecera_B": "05:30:00",
                "horario_cierre_cabecera_B": "23:30:00",
                "combinaciones": {"Plaza Miserere": {"ingreso_a_b": [1, 1],
                                                     "egreso_a_b": [3, 3]}, 
                                  "Lima": {"ingreso_a_b": [1, 1],
                                            "egreso_a_b": [3, 3]}, 
                                  "Peru": {"ingreso_a_b": [1, 1],
                                            "egreso_a_b": [3, 3]}}}}
        elif dia_semana == "sabado":
            linea = {"A": {
                "recorrido_A": ['San Pedrito', 'Flores', "-1", 'Carabobo', "-2", 
                                'Puan', "-3", 'Primera Junta', "-4",
                                'Acoyte', 'Rio de Janeiro', "-5", 'Castro Barros', 
                                "-6", 'Loria', "-7", 'Plaza Miserere', 'Pasco', 
                                "-8", 'Congreso', 'Saenz Peña', 'Lima', 'Piedras', 
                                "-9", 'Peru', 'Plaza de Mayo', "final"],
                "recorrido_B": ['Plaza de Mayo', 'Peru', 'Piedras', 'Lima', 
                                'Saenz Peña', "-1", 'Congreso', 'Alberti', "-2", 
                                'Plaza Miserere', "-3", 'Loria', 'Castro Barros', 
                                "-4", 'Rio de Janeiro', 'Acoyte', "-5", 
                                'Primera Junta', "-6", 'Puan', "-7", 'Carabobo', 
                                'Flores', 'San Pedrito', "final"],
                "horario_apertura_cabecera_A": "06:00:00", 
                "horario_cierre_cabecera_A": "23:59:00",
                "horario_apertura_cabecera_B": "06:00:00",
                "horario_cierre_cabecera_B": "23:59:00",
                "combinaciones": {"Plaza Miserere": {"ingreso_a_b": [1, 1],
                                                     "egreso_a_b": [3, 3]}, 
                                  "Lima": {"ingreso_a_b": [1, 1],
                                            "egreso_a_b": [3, 3]}, 
                                  "Peru": {"ingreso_a_b": [1, 1],
                                            "egreso_a_b": [3, 3]}}}}
        else:
            linea = {"A": {
                "recorrido_A": ['San Pedrito', 'Flores', "-1", 'Carabobo', "-2", 
                                'Puan', "-3", 'Primera Junta', "-4",
                                'Acoyte', 'Rio de Janeiro', "-5", 'Castro Barros', 
                                "-6", 'Loria', "-7", 'Plaza Miserere', 'Pasco', 
                                "-8", 'Congreso', 'Saenz Peña', 'Lima', 'Piedras', 
                                "-9", 'Peru', 'Plaza de Mayo', "final"],
                "recorrido_B": ['Plaza de Mayo', 'Peru', 'Piedras', 'Lima', 
                                'Saenz Peña', "-1", 'Congreso', 'Alberti', "-2", 
                                'Plaza Miserere', "-3", 'Loria', 'Castro Barros', 
                                "-4", 'Rio de Janeiro', 'Acoyte', "-5", 
                                'Primera Junta', "-6", 'Puan', "-7", 'Carabobo', 
                                'Flores', 'San Pedrito', "final"],
                "horario_apertura_cabecera_A": "08:00:00", 
                "horario_cierre_cabecera_A": "22:08:00",
                "horario_apertura_cabecera_B": "08:00:00",
                "horario_cierre_cabecera_B": "22:36:00",
                "combinaciones": {"Plaza Miserere": {"ingreso_a_b": [1, 1],
                                                     "egreso_a_b": [3, 3]}, 
                                  "Lima": {"ingreso_a_b": [1, 1],
                                            "egreso_a_b": [3, 3]}, 
                                  "Peru": {"ingreso_a_b": [1, 1],
                                            "egreso_a_b": [3, 3]}}}}
            
        df = pd.read_csv(f"sentido_viajes_A_x_min_{dia_semana}.csv")
        simulador(df, linea, capacidad_tren=845, meses="todos", 
                  frecuencia=3, sesgo_distancia=0.4, 
                  nivel_sesgo_distancia=20, nivel_sesgo_siguiente=20,
                  porcentaje_sin_pagar=0.03, probabilidad_no_subir=0.05, 
                  incremento_probabilidad_no_subir=[0.1, 100],
                  penalizacion_llenado_imperfecto=0.05, dia_semana=dia_semana)
    
    for dia in ["lunes", "martes", "miercoles", "jueves", "viernes", "sabado", "domingo", "feriado"]:
        t0 = timer()
        programa(dia)
        t1 = timer()
        print(dia, "completado en:", t1-t0, "s")
        
    
    # añadir un multiplicador para egresos en combinaciones, y ver si se puede HECHO
    # hacer alguna funcion para dar prioridad a estaciones cercanas o lejanas HECHO
    # hacer que el sesgo de la prioridad sea editable en su fuerza HECHO
    # hacer que las probabilidades de que una persona viaje sólo una estacion sean
    # muy bajas HECHO
    # añadir una penalizacion por llenado imperfecto que reduzca la barrera para
    # que los pasajeros lo vean mas lleno de lo que esta, y comiencen a quedarse 
    # en la estacion HECHO
    # ver de cambiar el sistema para calcular distancia entre estaciones, para hacerlo
    # completamente editable
    # añadir frecuencias distintas segun mes
    
    # segun https://www.buenosaires.gob.ar/noticias/macri-presento-los-nuevos-vagones-de-la-linea
    # la capacidad por vagon es de (133 parados + 36 sentados), con 5 vagones da un total de 845
    # la probabilidad de no subir aumenta 10% por cada 100 pasajeros por sobre la capacidad
    # teorica del tren